In [ ]:
import surfa 
import numpy as np
from surfa import Overlay
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import cuml
import cupy as cp  # cuML uses CuPy arrays (which are similar to NumPy but for GPU
import torch

from sklearn.cluster import DBSCAN
import numpy as np


In [3]:

print("CUDA Available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())

CUDA Available: True
Number of GPUs: 1


In [4]:
!nvidia-smi

Wed Jan  8 16:07:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40                     On  |   00000000:0A:00.0 Off |                    0 |
| N/A   30C    P8             34W /  300W |       3MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
lh_mean = surfa.load_overlay('rh.pial.H')

In [5]:
lh = surfa.load_mesh('rh.pial')

In [6]:
lh_mean.data>0

array([False, False, False, ...,  True,  True,  True])

In [7]:
mask = lh_mean.data > 0

# Apply the mask to the vertices (broadcasting)
masked_vertices = lh.vertices * mask[:, np.newaxis]

In [8]:
np.shape(masked_vertices)

(148696, 3)

In [ ]:
type(lh.space)

In [ ]:
np.shape(lh.vertices)

In [ ]:
np.shape(lh_mean.data)

In [ ]:


# Assume masked_vertices is your data, and you want to apply DBSCAN
masked_vertices_gpu = cp.array(masked_vertices)

# Apply DBSCAN from cuML
db_gpu = cuml.cluster.DBSCAN(eps=0.5, min_samples=100)
labels_gpu = db_gpu.fit_predict(masked_vertices_gpu)

# Convert the labels back to NumPy (if needed)
labels = cp.asnumpy(labels_gpu)

# Output the resulting labels
print(labels)


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

db = DBSCAN(eps=0.1, min_samples=50).fit(masked_vertices)  # Adjust `eps` and `min_samples` based on your data

# Get cluster labels
labels = db.labels_

# Visualize clusters (optional)
#import matplotlib.pyplot as plt
#plt.scatter(masked_vertices[:, 0], masked_vertices[:, 1], c=labels)
#plt.show()